In [213]:
#Spacey Load into Colab
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [214]:
#Imports  all posible imports used or to be used
#CNN,RNN imports 
%load_ext tensorboard
%tensorflow_version 2.X
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras.layers import Input,Embedding, Dense,LSTM, Conv2D,Conv1D, GlobalAveragePooling2D, MaxPooling2D,Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import string
import spacy
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#NLP LOAD SPACY


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.X`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [215]:
nlp = spacy.load("en_core_web_lg")

In [218]:
#DATA PATH
PATH = "/content/drive/My Drive/spotifydata/"

In [219]:
#ALL DATASET MOUNTS
global_csv = pd.read_csv(PATH+'global.csv')
data_csv = pd.read_csv(PATH+'data.csv')
final_csv = pd.read_csv(PATH+'final_csv.csv')
completed_csv = pd.read_csv(PATH+'Spotify_3Mil_Dataset.csv')
print("Global", global_csv.shape)
print("Data", data_csv.shape)
print("Final",final_csv.shape)
print("Completed",completed_csv.shape)

Global (72600, 22)
Data (3111889, 22)
Final (250053, 22)
Completed (3183212, 24)


In [220]:
completed_csv.drop_duplicates(subset=['track_id'],inplace=True)
completed_csv.shape

(20116, 24)

In [221]:
#Merge with Genres DB
real_final_csv = completed_csv.merge(final_csv,how='inner',on='track_name')
real_final_csv.shape

(498058, 45)

In [222]:
real_final_csv.columns

Index(['Unnamed: 0', 'region_x', 'position_x', 'month_x', 'day_x', 'year',
       'region.1', 'streams_x', 'artist_x', 'track_name', 'track_id',
       'duration_ms_x', 'time_signature_x', 'danceability_x', 'energy_x',
       'key', 'loudness_x', 'mode', 'speechiness_x', 'acousticness_x',
       'instrumentalness_x', 'liveness_x', 'valence_x', 'tempo_x',
       'position_y', 'artist_y', 'streams_y', 'url', 'Year', 'month_y',
       'day_y', 'region_y', 'danceability_y', 'energy_y', 'loudness_y',
       'speechiness_y', 'acousticness_y', 'instrumentalness_y', 'liveness_y',
       'valence_y', 'tempo_y', 'duration_ms_y', 'time_signature_y',
       'Artist Popularity', 'Genres_cleaned'],
      dtype='object')

In [228]:
columns = [ 'position_y', 'artist_y', 'streams_y', 'url', 'Year', 'month_y',
           'day_y', 'region_y', 'danceability_y', 'energy_y', 'loudness_y',
           'speechiness_y', 'acousticness_y', 'instrumentalness_y', 'liveness_y',
           'valence_y', 'tempo_y', 'duration_ms_y', 'time_signature_y']

In [229]:
real_final_csv.drop(columns=columns,index=1,inplace=True)
real_final_csv

KeyError: ignored

In [230]:
real_final_csv.rename(columns={'region_x':'region', 'position_x':'position',
                               'month_x':'month', 'day_x':'day','region.1':'region',
                               'streams_x':'streams','artist_x':'artist','duration_ms_x':'duration_ms',
                               'time_signature_x':'time_signature', 'danceability_x':'danceability',
                               'energy_x':'energy', 'loudness_x':'loudness','speechiness_x':'speechiness',
                               'acousticness_x':'acousticness',
                               'instrumentalness_x':'instrumentalness', 'liveness_x':'liveness',
                               'valence_x':'valence', 'tempo_x':'tempo'},inplace=True)

In [231]:
real_final_csv.shape

(498057, 26)

In [232]:
df_noduplicate = real_final_csv.drop_duplicates(subset=['track_id'])

In [233]:
df_noduplicates = df_noduplicate.reset_index()

In [234]:
completed_csv = completed_csv.reset_index()

In [235]:
col_drop = ['index','Unnamed: 0']
completed_csv.drop(columns=col_drop, axis=1, inplace=True)
df_noduplicates.drop(columns=col_drop, axis=1, inplace=True)

In [236]:
completed_csv

,region,position,month,day,year,region.1,streams,artist,track_name,track_id,duration_ms,time_signature,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,global,1,1,1,2017,global,3135625,The Weeknd,Starboy,5aAx2yezTd8zXrkmtKl66Z,230453,4,0.681,0.594,7,-7.028,1,0.2820,0.16500,0.000003,0.1340,0.535,186.054
1,global,2,1,1,2017,global,3015525,The Chainsmokers,Closer,7BKLCZ1jbUBVqRi2FVlTVw,244960,4,0.748,0.524,8,-5.599,1,0.0338,0.41400,0.000000,0.1110,0.661,95.010
2,global,3,1,1,2017,global,2545384,DJ Snake,Let Me Love You,4pdPtRcBmOSQDlJ3Fk945m,205947,4,0.476,0.718,8,-5.309,1,0.0576,0.07840,0.000010,0.1220,0.142,199.864
3,global,4,1,1,2017,global,2356604,Clean Bandit,Rockabye (feat. Sean Paul & Anne-Marie),5knuzwU65gJK7IF5yJsuaW,251088,4,0.720,0.763,9,-4.068,0,0.0523,0.40600,0.000000,0.1800,0.742,101.965
4,global,5,1,1,2017,global,2259887,Drake,One Dance,1xznGGDReH1oQq0xzbwXa3,173987,4,0.791,0.619,1,-5.886,1,0.0532,0.00784,0.004230,0.3510,0.371,103.989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20111,ph,97,12,24,2018,ph,42190,Maria Aragon,Da Best Ang Pasko Ng Pilipino,1xbJ8LIHgUclqlTx4cyEG9,274158,4,0.604,0.598,5,-6.334,1,0.0468,0.45900,0.000000,0.1070,0.493,135.747
20112,ph,54,12,31,2018,ph,62808,Karencitta,Cebuana - Muffin Remix,3K5iV064JlAqt9tYVZP2KF,284107,4,0.722,0.874,0,-4.282,0,0.0523,0.12500,0.060700,0.1890,0.682,98.022
20113,ph,57,12,31,2018,ph,62289,Jonas Blue,Mama,4UYYgQufzw5Ep8Vm4wMC61,184134,4,0.738,0.794,11,-4.118,0,0.0416,0.10100,0.000000,0.0533,0.566,104.014
20114,ph,69,12,31,2018,ph,53083,Cash Cash,Call You (feat. Nasri of MAGIC!),1gdMKFGWrI34ZTO8k7Hfg6,219451,3,0.494,0.767,10,-4.101,1,0.0510,0.12000,0.000000,0.3740,0.112,125.506


In [237]:
df_noduplicates

,region,position,month,day,year,region,streams,artist,track_name,track_id,duration_ms,time_signature,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Artist Popularity,Genres_cleaned
0,global,1,1,1,2017,global,3135625,The Weeknd,Starboy,5aAx2yezTd8zXrkmtKl66Z,230453,4,0.681,0.594,7,-7.028,1,0.2820,0.165,0.000003,0.134,0.535,186.054,63,"'canadian contemporary r&b','canadian pop','pop'"
1,global,60,5,3,2017,global,1137153,The Weeknd,Starboy,7MXVkk9YMctZqd1Srtv4MB,230453,4,0.678,0.588,7,-7.015,1,0.2760,0.141,0.000006,0.137,0.486,186.005,63,"'canadian contemporary r&b','canadian pop','pop'"
2,global,2,1,1,2017,global,3015525,The Chainsmokers,Closer,7BKLCZ1jbUBVqRi2FVlTVw,244960,4,0.748,0.524,8,-5.599,1,0.0338,0.414,0.000000,0.111,0.661,95.010,67,"'dance pop','edm','electropop','pop','tropical..."
3,lt,89,12,7,2018,lt,1131,Astrid S,Closer,0beQOSJ5aUkQky6XYlC8m6,183041,5,0.531,0.472,0,-7.347,0,0.0590,0.394,0.000000,0.139,0.445,127.480,67,"'dance pop','edm','electropop','pop','tropical..."
4,nz,71,10,26,2017,nz,9763,Six60,Closer,7wDjHY3hLrGt4NaXI7o7ck,196869,4,0.547,0.717,0,-6.546,1,0.2170,0.243,0.000011,0.148,0.621,97.884,67,"'dance pop','edm','electropop','pop','tropical..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5598,ph,72,9,7,2018,ph,37652,No Rome,Narcissist,4Zft1h1zRfkBqlAo4qEPqp,195502,4,0.543,0.682,3,-5.810,1,0.1120,0.219,0.000002,0.085,0.424,179.972,67,"'indie pop','indietronica','pinoy indie'"
5599,ph,56,9,21,2018,ph,45273,Erik Santos,Kung Akin Ang Mundo,0W2kzUI1K0tT0RpggJZ4Js,210046,4,0.636,0.397,5,-9.477,1,0.0349,0.834,0.000040,0.096,0.328,135.875,61,'opm'
5600,ph,32,10,5,2018,ph,79216,LANY,Taking Me Back,6A9mMKHYdeUJGvQDKChC0C,194811,4,0.689,0.570,5,-6.728,0,0.0419,0.040,0.000000,0.106,0.524,94.061,61,"'la pop','pop'"
5601,ph,33,10,5,2018,ph,74172,LANY,Let Me Know,7DMPq3XndRJaj6NTINsLOz,276197,4,0.696,0.391,11,-8.306,0,0.0825,0.264,0.000003,0.127,0.650,94.971,61,"'la pop','pop'"


In [ ]:
# TOP LINE SET UP DATASETS TO USEONLY DFNODUPLICATES AND COMPLETED_CSV#

In [238]:
completed_csv[['danceability','energy','key', 'loudness','mode', 'speechiness', 'acousticness','instrumentalness',	'liveness' , 'valence','tempo',	'duration_ms','time_signature']]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.681,0.594,7,-7.028,1,0.2820,0.16500,0.000003,0.1340,0.535,186.054,230453,4
1,0.748,0.524,8,-5.599,1,0.0338,0.41400,0.000000,0.1110,0.661,95.010,244960,4
2,0.476,0.718,8,-5.309,1,0.0576,0.07840,0.000010,0.1220,0.142,199.864,205947,4
3,0.720,0.763,9,-4.068,0,0.0523,0.40600,0.000000,0.1800,0.742,101.965,251088,4
4,0.791,0.619,1,-5.886,1,0.0532,0.00784,0.004230,0.3510,0.371,103.989,173987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20111,0.604,0.598,5,-6.334,1,0.0468,0.45900,0.000000,0.1070,0.493,135.747,274158,4
20112,0.722,0.874,0,-4.282,0,0.0523,0.12500,0.060700,0.1890,0.682,98.022,284107,4
20113,0.738,0.794,11,-4.118,0,0.0416,0.10100,0.000000,0.0533,0.566,104.014,184134,4
20114,0.494,0.767,10,-4.101,1,0.0510,0.12000,0.000000,0.3740,0.112,125.506,219451,3


# Functions to get features from track ID

In [239]:
def getfeatures(id,df):
  """Gets Features From track for Neural Network Model"""
  features = df.copy()
  if id in (features['track_id'].values):
    features = features[features['track_id']==id]
    features = features[['danceability','energy','key','loudness','mode', 
                         'speechiness', 'acousticness','instrumentalness',
                         'liveness' , 'valence','tempo']]
    return features
  else:
    raise Exception('Track ID Not Found')

In [240]:
song_feats = getfeatures('6G3tTLzjROxw2KpZjUvICy',completed_csv)

In [241]:
song_feats

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
20115,0.709,0.687,6,-5.695,1,0.173,0.0126,0.000007,0.17,0.62,104.876


# NLP Processing model

In [ ]:
## Following Code by Santiago 

In [242]:
#Tokenizing Function 
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.lower().strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [ ]:
df_noduplicates.head()

In [ ]:
df_noduplicates

In [245]:
#Function to create Data string given the columns/features 
def create_data(df,feature_col):
  data = ''
  for i in feature_col:
    data += df[i]+' '
  return data

In [246]:
#Gives Features and create data string
feature_col = ['track_name','artist','Genres_cleaned']
data = create_data(df_noduplicates,feature_col)

In [ ]:
#TFIDF Term Frequency Document Inverse frequency Document term matrix
tfidf = TfidfVectorizer(ngram_range=(1,2),
                        max_df=.97,
                        min_df=.02,
                        tokenizer=tokenize)

# Create a vocabulary and get word counts per document
dtm1 = tfidf.fit_transform(data) # 

In [83]:
#Get Nearest Neighbor
# Fit on DTM
near_neighbor = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
near_neighbor.fit(dtm1)

/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [84]:
#Test Song 
song= [ """Billie Eilish Bad Guy"""]

In [85]:
def findsimilar(song):
  transform = tfidf.transform(song)
  similar = near_neighbor.kneighbors(transform.todense())
  return similar
  

In [86]:
def print_similar(similar):
  arr = similar[1][0]
  for i, song in enumerate(arr):
    print("Similar Song: ",data[arr[i]])

In [87]:
similarsong = findsimilar(song)

In [88]:
print_similar(similarsong)

Similar Song:  Winter Wonderland / Don't Worry Be Happy Pentatonix 'a cappella' 
Similar Song:  You're The One That I Want - From “Grease” John Travolta 'hollywood' 
Similar Song:  Veneno Anitta 'baile pop','funk carioca','funk das antigas','pagode baiano' 
Similar Song:  Off to See the World Lukas Graham 'danish pop','pop','scandipop' 
Similar Song:  Must Be Santa Bob Dylan 'album rock','classic rock','country rock','folk','folk rock','mellow gold','rock','roots rock','singer-songwriter' 


In [ ]:
similarsong

# Target Setups For NN Models

In [248]:
completed_csv = completed_csv.reset_index()

In [249]:
completed_csv.drop('index',axis=1,inplace=True)

In [ ]:
completed_csv.head()

In [251]:
#Label Encoders
le = LabelEncoder()
#Features 
X_features = completed_csv[['danceability','energy','key','loudness','mode', 'speechiness', 'acousticness','instrumentalness',	'liveness' , 'valence','tempo']]
target = completed_csv['track_id']
#Encoded target
encoded_target = le.fit_transform(target)

In [252]:
#TEst Encodign
le.inverse_transform(encoded_target)

array(['5aAx2yezTd8zXrkmtKl66Z', '7BKLCZ1jbUBVqRi2FVlTVw',
       '4pdPtRcBmOSQDlJ3Fk945m', ..., '4UYYgQufzw5Ep8Vm4wMC61',
       '1gdMKFGWrI34ZTO8k7Hfg6', '6G3tTLzjROxw2KpZjUvICy'], dtype=object)

In [253]:
#Reshaping for CNN and model SPlit
#Regular Shape  Model Splite
X_train_feat,X_val_feat,y_train_feat,y_val_feat = train_test_split(X_features,encoded_target,test_size=0.2)
X_train_feat.shape,y_train_feat.shape,X_val_feat.shape,y_val_feat.shape


((16092, 11), (16092,), (4024, 11), (4024,))

In [254]:
#Split with Reshape
X_feat_reshaped = X_features.values.reshape(X_features.shape[0],X_features.shape[1],1)
X_train_reshaped,X_val_reshaped,y_train_reshaped,y_val_reshaped = train_test_split(X_feat_reshaped,encoded_target,test_size=0.2)
X_train_reshaped.shape,y_train_reshaped.shape,X_val_reshaped.shape,y_val_reshaped.shape

((16092, 11, 1), (16092,), (4024, 11, 1), (4024,))

# Features KNN 

In [255]:
nn = NearestNeighbors(n_neighbors=10,algorithm='kd_tree')
nn.fit(X_features,encoded_target)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [256]:
#Test Song
#Find songs similar features
sim_feat = nn.kneighbors(song_feats)

In [144]:
sim_feat

(array([[0.        , 0.60651431, 0.65143259, 0.66617708, 0.86648304,
         0.95041417, 0.96578942, 1.02931938, 1.03985101, 1.04725312]]),
 array([[20115, 17715, 15616, 15618,  8140,   363, 19718, 15961,  9615,
         18621]]))

In [ ]:
preddf = pd.DataFrame()
arr = sim_feat[1][0]
for i,_ in enumerate(arr):
  preddf = preddf.append(X_features[X_features.index == arr[i]])
preddf

In [319]:
# Function for Unit 3 to get the track id's from the dataset
data = np.array([])
for i,_ in enumerate(arr):
  data = np.append(data,completed_csv['track_id'][completed_csv.index==arr[i]])

In [321]:
data[0]

'6G3tTLzjROxw2KpZjUvICy'

In [266]:
import joblib

In [268]:
joblib.dump(nn, 'knn.joblib', compress=True) # KNN Drop

['knn.joblib']

In [270]:
completed_csv.to_csv('herokuspotify.csv',index=False)

#Simple NeuralNetwork

In [177]:
from sklearn.preprocessing import StandardScaler

In [178]:
ss = StandardScaler()

In [322]:

encoded_target.astype('float32')

array([14375., 18477., 12479., ..., 11601.,  4394., 16201.], dtype=float32)

In [ ]:
dense = Dense(10, activation='relu', kernel_initializer='he_normal')
output = Dense(1, activation='sigmoid')(dense)
model2 = Model(inputs=in_layers, outputs=output)

In [331]:

model = Sequential(
      [
       Dense(32, activation='relu', input_dim=11),
       Dense(32, activation='relu'),
       Dense(1, activation='softmax')

      ]
  )
model.compile(optimizer='adam', loss='sparse_crossentropy', metrics=['accuracy'])

In [332]:
model.fit(X_train_feat,y_train_feat,epochs=1,validation_data=(X_val_feat,y_val_feat))

503/503 [==============================] - 2s 3ms/step - loss: -153947.7188 - accuracy: 0.0000e+00 - val_loss: -150979.5156 - val_accuracy: 2.4851e-04


In [333]:
encoded_target[1]

18477

In [334]:
predt = model.predict(X_train_feat)

In [335]:
predt.astype('int')

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [336]:
le.inverse_transform([int(test)])

array(['3OJq3VFyNrU0INcTwPyjbD'], dtype=object)

# LSTM Model

In [72]:
X_features.shape

(20116, 11)

In [ ]:
#LSTM Model
model_play = Sequential()
model_play.add(LSTM(128, input_shape=(encoded_target))))
model_play.add(Dense(len(chars),activation='softmax'))
model_play.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [165]:
X_feat_reshaped.shape

(20116, 11, 1)

# CNN Model

In [160]:
len(encoded_target)

20116

In [162]:
model = Sequential()
model.add(Conv1D(32, 2, activation="relu", input_shape=(11, len(encoded_target))))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(1,activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 10, 32)            1287456   
_________________________________________________________________
flatten_4 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                20544     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 1,308,065
Trainable params: 1,308,065
Non-trainable params: 0
_________________________________________________________________


In [164]:
model.fit(X_features,encoded_target)

ValueError: ignored

In [152]:
songfeatsreshaped = song_feats.values.reshape(song_feats.shape[0],song_feats.shape[1],1)

In [153]:
preds = model.predict(songfeatsreshaped)

In [151]:
predreshape = preddf.values.reshape(preddf.values.shape[0],preddf.values.shape[1],1)
model.predict(predreshape)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [158]:
test = int(preds)

In [159]:
le.inverse_transform([test])

array(['000xYdQfIZ4pDmBGzQalKU'], dtype=object)

# Experiment Zone

In [111]:
nn = NearestNeighbors(n_neighbors=5,algorithm='auto')

In [112]:
nn.fit(X_features,X_features)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [113]:
song_feats

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
20115,0.709,0.687,6,-5.695,1,0.173,0.0126,0.000007,0.17,0.62,104.876


In [115]:
most_similar = nn.kneighbors(song_feats)

In [116]:
most_similar

(array([[0.        , 0.60651431, 0.65143259, 0.66617708, 0.86648304]]),
 array([[20115, 17715, 15616, 15618,  8140]]))

In [ ]:
X_features[553]

In [ ]:
most_similar = nn.kneighbors(song_feats)

In [ ]:
le.inverse_transform(encoded_target[[most_similar[1][0][1]]])

array(['Pasado y Presente'], dtype=object)